In [ ]:
import json 
import logging ; import os 
import subprocess 
import time 

from tpu_survival import TPUServival
import nbimporter 
from experiments import experiments 


In [ ]:
# khai báo biến project = 1 chuỗi rỗng được sử dụng  để lưu trữ tên dự án 
project = ''
# klhai báo một biến location có gái trị là us-central1-f . Biến này có thể được 
# sử dụng để xác định vị trí  của máy chủ 
location = 'us-central1-f'


# gọi phương thức getLogger của modun logging để tạo một đối tượng logger có tên là logger 
# đối tượng này sẽ được sử dụng để ghi nhật ký các sự kiện trong chương trình.
logger = logging.getLogger()

# 3. Cài đặt mức độ log 
# Thiết lập mức độ log của logger thành INFO các message log có mức độ cao hưn hoặc bằng INFO
# sẽ được ghi lại 
logger.setLevel(logging.INFO)

# 4. Thiết lập FileHandler:
# Tạo một FileHandler  đối tượng dùng để ghi log vào file . FileHandler 
# được khởi tạo với đường dẫn đến file log logs/overrunner.log.
fh = logging.FileHandler('logs/overrunner.log')

# Thiết lập mức độ log của fileHandler thành INFO tương tự như logger chính 
fh.setLevel(logging.INFO)

# 5. Thiết lập StreamHandler:
# Tạo một streamHandler đối tượng dùng để ghi log ra console 
sh = logging.StreamHandler()
# Thiết lập mức độ log cua streamHandler thành INFO tuonwg tự như trên 
sh.setLevel(logging.INFO)

# 6. Định dạng message log:
# Gọi lớp Formatter của mô-đun logging với tham số là '%(asctime)s:%(name)s:%(levelname)s:%(message)s'
#  để tạo một đối tượng formatter có tên là formatter.
#  Đối tượng formatter này sẽ định dạng các bản ghi nhật ký theo mẫu cho trước,
#  bao gồm thời gian, tên logger, mức độ nhật ký, và nội dung nhật ký.
formatter = logging.Formatter('%(asctime)s:%(name)s:%(levelname)s:%(message)s') # Add timestamp to file logging
# Thiết lập Formatter cho fileHandler đảm bảo mesage log ghi vào file được chỉ định theo format đã cấu hình 
fh.setFormatter(formatter)

# 7. Thêm Handler vào logger:
# thêm FileHandler vào logger. Bây giờ, bất kỳ message log nào được ghi bằng logger này
# sẽ được ghi vào file logs/overrunner.log.
logger.addHandler(fh)

# thêm StreamHandler vào logger. Bây giờ, message log cũng sẽ được in ra console.
logger.addHandler(sh)

In [ ]:
# thiết lập đường dẫn lưu trữ backup 
# lưu trữ đường dẫn đến 1 backup trên google Cloud Storage để lưu trữ các bản backup 
backup_path = "gs://connors-models/backup"

# tạo một thư mục log nếu chưa tồn tại 
# kiểm tra xem thư mục log có tồn tại chưa 
if not os.path.exists("log"):
    # nếu không tạo 1 thư mục log 
    os.mkdir("log")

#  Khởi tạo runners mới hoặc phục hồi runners từ trạng thái đã lưu:
# kiểm tra xem file state.json đã tồn tại trong thư mục log hay chưa 
if not os.path.exists("log/state.json"):
    # nêu chưa tồn tại 
    # tạo một danh sacgs cácc đối tươgj TFSurvival mới , mỗi đối tượng đại diện chp một runner
    # tức trình chạy trên TPU . project, location: Có thể là thông tin dự án và vùng tính toán trên Google Cloud.
    # params tham số cấu hình cho runner 
    runners = [TPUServival(project=project, location=location, id=i, params=ex) for i , ex in enumerate(experiments)]

else: # nếu đã tồn tại 
    # mở file state.json và gán nó cho f 
    with open("log/state.json", "r") as f: 
        # Đcọ dữ liệu từ file json và tạo lại các đối tượng TPUSurvival từ trạng thái đã lưu 
        runners = [TPUServival(d=state) for state in json.load(f)]


# tạo một danh sách các đối tuonwgj TPUSurvival mới bằng cách sao chép lại danh sách ban đầu 
all_runners = runners.copy() # để đảm bảo tất acr TPU sẽ bị xóa bất kể điều gì 


# Thiết lập phương thức lưu trữ trạng thái các đối tượng runner vào fiel json 
# nhận đầu vào là 1 dnah sách các trạng thái runner được sử dụng để chạy các thử nghiệm trê TPU 
# Mỗi runner đại diện cho một tiến trình chạy trên TPU. Nó chịu trách nhiệm thực hiện các bước sau:
# Tải mô hình học máy đã được đào tạo.
# Chuẩn bị dữ liệu đầu vào cho mô hình.
# Chạy mô hình trên dữ liệu đầu vào.
# Thu thập và lưu trữ kết quả.
def save(runners):
    # khởi tạo 1 danh sách rỗng để lưu trữ danh sách các trạng thái của runner 
    state = []
    # lặp qua các runner 
    for r in runners:
        # gọi phương thức dump_dict để lấy accs trạng thái của runners dưới 
        # dnagj từ điển và thêm vào dnah sách lưu trữ 
        state.append(r.dump_dict())
    
    # mở các file lưu trữ trạng thaí của runners trong tệp log với chế độ ghi 
    with open("log/state.json", "w") as f:
        # sử dụng module json để ghi các danh sách state vào file json 
        json.dump(state, f)
        

In [ ]:
# # lấy thời gian hiện tại gán cho running  
running = time.time()

# Xây dựng một khối xử lý ngoại lệ trong quá trình thực thi 
# là một vòng lặp chính thực hiện quản lý các runner cho đến khi không còn runner nào hoạt 
# động (mỗi runners là một trình chạy thử nghiệm TPU)

# 1 Vòng lặp chính Try : 
try: 
    # Xây dựng vòng lặp while cho đến khi danh sách runners trống 
    while len(runners) > 0: 
        # duyệt qua các phần tử trong danh sách runner gán các phần tử cho biến ts
        for ts in runners: 
            # kiểm tra xem công việc của runners (ts) đã hoàn thành hay chưa 
            if ts.done: 
                # ta loại bỏ runners đó 
                runners.remove(ts)
                # sau đó ta tiếp tục thực thi 
                continue 
            # nếu công việc của runners đó chưa hoàn thanh ta cập nhật trạng thái cho 
            # runners đó 
            ts.update_state()
            # sau đó ghi thông tin trạng thái của trình chạy thử nghiệm runner bảo gồm 
            # prefix , trạng thái TPU và trạng thái chạy tác vụ , vào log 
            logging.info("{} - TPU State: {} - Process Running: {}".format(ts.prefix, ts.state, ts.task_running))

            # kiểm tả xem TPU đã được tạo hay chưa 
            if not ts.created: 
                # ghi thông tin đang tạo TPU vào nhật ký log 
                logging.info("{} -Creating TPU" .format(ts.prefix))
                # sau đó gọi phương thức create của runner để tạo TPU 
                ts.reate()

            # kiểm tả xem TPU đã sẵn sàng cho trình chạy thử nghiệm và chưa có tác vụ chạy hay chưa 
            if ts.state == "READY" and not ts.task_running: 
                # ghi một thông tin bắt đầu chạy tác vụ vào log 
                logging.info("{} -Starting Task" .format(ts.prefix))
                # gọi tác vụ run task của runner để bắt đầu chạy tác vụ Trên TPU 
                ts.run_task()

            # Kiểm tra tragj thái kết thúc của tác vụ 
            # khởi tạo biến return code để lưu trữ mã trả về của tác vụ 
            returncode = None 
            # kiểm tra xem tác vụ có đang chạy 
            if ts.task_running: 
                # kiểm tra xem tác vụ đã hoàn thành chưa 
                ts.current_process.poll()
                # Và lấy mã trả về của tác vụ 
                returncode = ts.current_process.returncode 

            # Kiểm tra xem có mã trả về của tác vụ hay chưa 
            if returncode is not None: 
                # Ghi thông tin quá trình huấn luyện đã kết thúc với mã trả về 
                logging.info("{} - Training process terminated with code: {}." .format(ts.prefix, returncode))

                # kiểm tra xem mã có trả về == 0 tức thành cồng 
                if returncode == 0 :
                    #   Thực hiện các tác vụ dọn dẹp sau khi hoàn thành, có thể bao gồm xóa TPU
                    ts.delete()
                    # và đánh dấu runner đã hoàn thành
                    ts.done = True
                    #  Xóa runner khỏi danh sách.
                    runners.remove(ts)
                    # Tiếp tục sang runner tiếp theo trong vòng lặp.
                    continue

            # Xử lý tác vụ bị gián đoạn:
            # kiểm tra xem tác vụ đang chạy nhưng ở trạng thái TPU chưa sẵn sàng 
            if ts.state != "READY" and ts.task_running:
                # ghi thông tin tác vụ bị gián đoạn Preempted vào log 
                logging.info("{} - Preempted".format(ts.prefix))
                # xóa TPU đang được sử dụng bởi runner này 
                ts.delete()
                # kết thúc tác vụ đang chạy trên TPU 
                ts.kill_current_task()


            # Lưu checkpoint mỗi ngày 
            # kiểm tra xem runner đã chạy được hơn 24 h 
            if ts.running_time > 60*60*24:
                # ghi thông tin đang tạo bản backup vào log 
                logging.info("Backing up {}".format(ts.prefix))
                #  Sử dụng lệnh gsutil để sao chép thư mục chứa mô hình (ts.params["model_dir"])
                # lên Google Cloud Storage (GCS) với tên file bao gồm prefix, 
                # số thứ tự lưu (current_save) và thời gian.
                subprocess.call(["gsutil", "cp", "-r", ts.params["model_dir"],
                    os.path.join(backup_path, ts.params["model_dir"].split("/")[-1] + "-" + str(ts.current_save))])
                # sau đó tăng số thứ tự lưu cho lần lưu tiếp theo 
                ts.current_save += 1
        
        # Lưu trạng thái mỗi giờ 
        # Kiểm tra xem đã hơn 1 giờ kể từ lần lưu trạng thái trước.
        if time.time() - running > 60 *60 :
            # Ghi thông tin đang lưu trạng thái vào log.
            logging.info("Saving state")
            # Gọi hàm save để lưu trạng của cac runner vào file json 
            save(runners)
            # Cập nhật thời gian lưu trạng thái gần nhất.
            running = time.time()

        # nghĩ 30s để tránh lãng tài nguyên và hạn chế truy vấn trạng thái TPU quá thường xuyên 
        time.sleep(30)

    # Kết thúc vòng lặp
    logging.info("All runners done."): # Ghi thông tin tất cả runner đã hoàn thành công việc vào log.

# Xử lý ngoại lệ 
# Khối except này sẽ bắt bất kỳ ngoại lệ nào xảy ra trong vòng lặp chính.
except Exception as e:
    #  Ghi thông tin lỗi vào log để hỗ trợ gỡ lỗi.
    logging.error("Error occured in check loop: {}".format(str(e)))


#  Thực hiện lệnh cuối cùng (finally):
# Khối finally sẽ luôn được thực hiện, bất kể có ngoại lệ xảy ra hay không, 
# nhằm đảm bảo chương trình kết thúc một cách gọn gàng.
finally:
    # Tăt và lưu trạng thái 
    # Ghi thông tin đang tắt tất cả các tiến trình vào log.
    logging.info("Shutting everything down...")
    #  Lặp qua tất cả các runner (bao gồm cả những runner đã hoàn thành hoặc bị lỗi).
    for ts in all_runners:
        try:
            # : Cố gắng xóa TPU đang được sử dụng bởi runner.
            ts.delete()
        except Exception as e:
            # : Nếu xảy ra lỗi khi xóa TPU, ghi thông tin lỗi vào log.
            logging.error(e)
        try:
            # Cố gắng kết thúc tác vụ đang chạy trên TPU (nếu có).
            ts.kill_current_task()
        except Exception as e: #  Nếu xảy ra lỗi khi kết thúc tác vụ, ghi thông tin lỗi vào log.
            logging.error(e)
    # Gọi hàm save để lưu trạng thái của tất cả các runner vào file JSON, giúp có thể khôi phục lại trạng thái nếu cần thiết. 
    save(all_runners)
